In [1]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
import time

## Assemble dataset for ML model

The aim here is to create a dataset where the predictand is whether there was an ignition at a location (1) or not (0) for Europe. 

Europe is divided into a 10 x 10 km grid. We have a list of historical observed ignition points. 

The static predictors are: 
- Road measurement (how many meters of road within grid)
- Powerline measurement (how many meters of powerline within grid)
- Elevation (mean in 5 x 5 grid cell)
- Elevation (max in 5 x 5 grid cell)
- Land cover

The weather predictors (daily resolution) are:
- 12 UTC temperature
- 12 UTC relative humidity
- 12 UTC wind speed
- Total daily precipitation
- Weekly total precipitation
- Weekly temperature mean 




In [2]:
folder = 'ML-data-Central-Europe-gridded'

In [3]:
fire_season = True

if fire_season == True:
    # months to remove
    months = [1, 2, 3, 4, 5, 11, 12]
    season_name = 'fire-season'
else: 
    # months to remove
    months = [6, 7, 8, 9, 10]
    season_name = 'out-of-season'

### 1. Open list of selected points

In [4]:
path = "O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/" + folder + "/"

savepath = "O:/Climate-and-Energy-Policy/CERM/Projects/Wildfire/Data/" + folder + "/"

In [5]:
df_all = pd.read_csv(path + '/selected_points_'+ season_name + '.csv')

In [6]:
df_all

,id,date,ignit
0,3198,2016-09-13,1
1,3198,2018-09-17,1
2,3365,2019-07-30,1
3,3859,2022-08-31,1
4,4014,2018-07-15,1
...,...,...,...
11271,6543,2022-07-24,0
11272,15697,2019-07-23,0
11273,15680,2020-08-05,0
11274,18599,2021-08-11,0


### Static predictors

In [7]:
static = gpd.read_file(path + 'combined-static/static_predictors.shp')

In [8]:
static

,id,mean_elev,north,east,south,west,slope30,road_den,pline_den,urban,forest,agricultur,irrigated,shrubland,grassland,other,geometry
0,42.0,7.067822,0.379731,0.130583,0.379655,0.108954,0.000000,5809.525933,0.000955,0.594899,0.000000,0.106168,0.0,0.204033,0.000000,0.094899,"POLYGON ((3809789.394 3135937.078, 3801278.507..."
1,43.0,1.386238,0.389989,0.124017,0.348513,0.137362,0.000000,2271.961052,0.000278,0.146769,0.011090,0.729094,0.0,0.045830,0.049791,0.017427,"POLYGON ((3809789.394 3135937.078, 3809789.394..."
2,44.0,6.592371,0.378674,0.141092,0.354650,0.125573,0.000009,863.000000,0.000364,0.007558,0.000000,0.895080,0.0,0.004742,0.092620,0.000000,"POLYGON ((3809789.394 3125937.078, 3809789.394..."
3,45.0,19.688050,0.472356,0.118029,0.266595,0.143007,0.000000,1210.000000,0.000394,0.067553,0.031241,0.856262,0.0,0.000000,0.044944,0.000000,"POLYGON ((3809789.394 3115937.078, 3809789.394..."
4,46.0,34.843144,0.514495,0.118132,0.241339,0.126065,0.003554,1126.396554,0.000216,0.000000,0.003467,0.979889,0.0,0.000000,0.016644,0.000000,"POLYGON ((3809789.394 3105937.078, 3809789.394..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17338,34121.0,0.106896,0.558465,0.101036,0.223603,0.116264,0.010259,100.000000,0.000124,0.010710,0.000000,0.030790,0.0,0.000000,0.000000,0.958501,"POLYGON ((5849789.394 2645937.078, 5849789.394..."
17339,34122.0,-0.007136,0.603465,0.103113,0.192992,0.099043,0.006311,10.841880,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.034902,0.965098,"POLYGON ((5851221.243 2645937.078, 5852155.049..."
17340,34123.0,0.851959,0.371575,0.167176,0.288838,0.171623,0.004255,0.217285,0.000291,0.000000,0.061925,0.000000,0.0,0.307502,0.399151,0.231423,"POLYGON ((5852155.049 2635937.078, 5853088.856..."
17341,34124.0,0.069657,0.368346,0.151503,0.311709,0.168561,0.003831,581.870867,0.000115,0.029456,0.039572,0.000000,0.0,0.077655,0.166915,0.686403,"POLYGON ((5853088.856 2625937.078, 5853537.062..."


In [9]:
static = static.drop(columns = ['geometry'])

In [10]:
df_all = df_all.set_index('id').join(static.set_index('id'))

In [11]:
df_all

,date,ignit,mean_elev,north,east,south,west,slope30,road_den,pline_den,urban,forest,agricultur,irrigated,shrubland,grassland,other
id,,,,,,,,,,,,,,,,,
3198.0,2016-09-13,1,62.071857,0.405813,0.105250,0.347762,0.141175,0.000013,1926.000000,0.000321,0.163200,0.373700,0.09840,0.0,0.290800,0.045400,0.028500
3198.0,2018-09-17,1,62.071857,0.405813,0.105250,0.347762,0.141175,0.000013,1926.000000,0.000321,0.163200,0.373700,0.09840,0.0,0.290800,0.045400,0.028500
3365.0,2019-07-30,1,79.004539,0.381919,0.111675,0.370962,0.135444,0.000106,1807.000000,0.000431,0.298500,0.341600,0.11980,0.0,0.217900,0.009100,0.013100
3859.0,2022-08-31,1,31.601013,0.377481,0.127556,0.348756,0.146206,0.000000,1506.000000,0.000292,0.018100,0.147200,0.58580,0.0,0.002600,0.065800,0.180500
4014.0,2018-07-15,1,28.120051,0.408975,0.116569,0.357025,0.117431,0.000000,1203.000000,0.000099,0.071400,0.577100,0.00620,0.0,0.279300,0.063000,0.003000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6543.0,2022-07-24,0,114.303181,0.383183,0.143425,0.323636,0.149761,0.000180,2859.923089,0.000550,0.147657,0.345253,0.38769,0.0,0.011303,0.073058,0.035039
15697.0,2019-07-23,0,463.010280,0.434444,0.130950,0.328050,0.106556,0.091744,852.446647,0.000235,0.000400,0.571500,0.24750,0.0,0.000000,0.180600,0.000000
15680.0,2020-08-05,0,306.004903,0.470794,0.132175,0.277175,0.119856,0.002794,2197.943859,0.000522,0.114800,0.324200,0.53600,0.0,0.000000,0.024500,0.000500


### Append weather predictors to each row (Date / ID)


In [12]:
temp_file = pd.read_csv(path + 'weather/temp-2008-2023.csv')
print('temp lodaded')

wind_file = pd.read_csv(path + 'weather/wind-2008-2023.csv')
print('wind lodaded')

rhum_file = pd.read_csv(path + 'weather/rhum-2008-2023.csv')
print('rhum lodaded')

prcp_file = pd.read_csv(path + 'weather/prcp-2008-2023.csv')
print('prcp lodaded')

temp lodaded
wind lodaded
rhum lodaded
prcp lodaded


In [13]:
temp_file

,Unnamed: 0.1,Unnamed: 0,id,geometry,centre,i,j,lon,lat,fwi,...,20231221,20231222,20231223,20231224,20231225,20231226,20231227,20231228,20231229,20231230
0,0,0,42.0,"POLYGON ((3809789.3935 3135937.0785, 3801278.5...",POINT (2.6450584595235505 51.113263852620676),326,239,2.6,51.1,3.235292,...,10.641083,10.621246,12.226654,12.117035,9.785583,10.418793,11.137787,10.728119,9.480316,9.080719
1,1,1,43.0,"POLYGON ((3809789.3935 3135937.0785, 3809789.3...",POINT (2.630991811564933 51.05737065251798),326,239,2.6,51.1,3.235292,...,10.641083,10.621246,12.226654,12.117035,9.785583,10.418793,11.137787,10.728119,9.480316,9.080719
2,2,2,44.0,"POLYGON ((3809789.3935 3125937.0785, 3809789.3...",POINT (2.6602542412414905 50.9677716283446),327,240,2.7,51.0,4.925437,...,10.535614,10.705231,12.304779,12.191254,9.865662,10.330902,11.143646,10.843353,9.410004,9.033844
3,3,3,45.0,"POLYGON ((3809789.3935 3115937.0785, 3809789.3...",POINT (2.67071261431574 50.878474782441245),327,241,2.7,50.9,5.180570,...,10.412567,10.609528,12.234467,12.015472,9.830505,10.151215,10.934662,10.595306,9.195160,8.891266
4,4,4,46.0,"POLYGON ((3809789.3935 3105937.0785, 3809789.3...",POINT (2.694270268280491 50.82205354071462),327,242,2.7,50.8,5.652404,...,10.381317,10.603668,12.212982,11.904144,9.898865,10.063324,10.786224,10.454681,9.066254,8.807282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17338,17338,17338,34121.0,"POLYGON ((5849789.3935 2645937.0785, 5849789.3...",POINT (29.664807679034908 45.12196437236942),596,299,29.6,45.1,27.098641,...,8.705536,5.689606,6.283295,12.925629,13.910583,13.174652,9.969818,10.235931,6.704925,10.172516
17339,17339,17339,34122.0,"POLYGON ((5851221.243018672 2645937.0785, 5852...",POINT (29.643086152904413 45.04794852856215),596,300,29.6,45.0,28.801920,...,8.764130,5.857575,6.277435,12.499847,12.633240,12.694183,9.717865,9.771088,6.765472,9.432282
17340,17340,17340,34123.0,"POLYGON ((5852155.049308276 2635937.0785, 5853...",POINT (29.617702771615733 44.961449014257845),596,300,29.6,45.0,28.801920,...,8.764130,5.857575,6.277435,12.499847,12.633240,12.694183,9.717865,9.771088,6.765472,9.432282
17341,17341,17341,34124.0,"POLYGON ((5853088.85559788 2625937.0785, 58535...",POINT (29.590975126515 44.87788075605745),596,301,29.6,44.9,31.161872,...,8.865692,5.976715,6.400482,12.226410,11.764099,12.319183,9.573334,9.571869,6.908050,8.879547


In [14]:
# Daily temperature
temp_file['id'] = temp_file.id.astype('int32')
temp_file = temp_file.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0', 'geometry', 'centre', 'i', 'j', 'lon', 'lat', 'fwi'])
temp_file = temp_file.set_index(['id'])

In [15]:
# Temperature mean
temp_file_mean = temp_file.copy()
temp_file_mean = temp_file_mean.rolling(7, axis = 1).mean()

C:\Users\ophme\AppData\Local\Temp\ipykernel_8312\672204434.py:3: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  temp_file_mean = temp_file_mean.rolling(7, axis = 1).mean()


In [16]:
# Daily wind
wind_file['id'] = wind_file.id.astype('int32')
wind_file = wind_file.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0','geometry', 'centre', 'i', 'j', 'lon', 'lat', 'fwi'])
wind_file = wind_file.set_index(['id'])

In [17]:
# Daily relative humidity
rhum_file['id'] = rhum_file.id.astype('int32')
rhum_file = rhum_file.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0', 'geometry', 'centre', 'i', 'j', 'lon', 'lat', 'fwi'])
rhum_file = rhum_file.set_index(['id'])

In [18]:
# Daily precipitation
prcp_file['id'] = prcp_file.id.astype('int32')
prcp_file = prcp_file.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0', 'geometry', 'centre', 'i', 'j', 'lon', 'lat', 'fwi'])
prcp_file = prcp_file.set_index(['id'])

In [19]:
# Weekly precipitation
prcp_file_sum = prcp_file.copy()
prcp_file_sum = prcp_file_sum.rolling(7, axis = 1).sum()

C:\Users\ophme\AppData\Local\Temp\ipykernel_8312\2084339385.py:3: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  prcp_file_sum = prcp_file_sum.rolling(7, axis = 1).sum()


In [20]:
# Monthly precipitation
prcp_file_long_sum = prcp_file.copy()
prcp_file_long_sum = prcp_file_sum.rolling(28, axis = 1).sum()

C:\Users\ophme\AppData\Local\Temp\ipykernel_8312\4140565408.py:3: FutureWarning: Support for axis=1 in DataFrame.rolling is deprecated and will be removed in a future version. Use obj.T.rolling(...) instead
  prcp_file_long_sum = prcp_file_sum.rolling(28, axis = 1).sum()


In [21]:
df_all['date'] = pd.to_datetime(df_all.date)

In [22]:
df_all

,date,ignit,mean_elev,north,east,south,west,slope30,road_den,pline_den,urban,forest,agricultur,irrigated,shrubland,grassland,other
id,,,,,,,,,,,,,,,,,
3198.0,2016-09-13,1,62.071857,0.405813,0.105250,0.347762,0.141175,0.000013,1926.000000,0.000321,0.163200,0.373700,0.09840,0.0,0.290800,0.045400,0.028500
3198.0,2018-09-17,1,62.071857,0.405813,0.105250,0.347762,0.141175,0.000013,1926.000000,0.000321,0.163200,0.373700,0.09840,0.0,0.290800,0.045400,0.028500
3365.0,2019-07-30,1,79.004539,0.381919,0.111675,0.370962,0.135444,0.000106,1807.000000,0.000431,0.298500,0.341600,0.11980,0.0,0.217900,0.009100,0.013100
3859.0,2022-08-31,1,31.601013,0.377481,0.127556,0.348756,0.146206,0.000000,1506.000000,0.000292,0.018100,0.147200,0.58580,0.0,0.002600,0.065800,0.180500
4014.0,2018-07-15,1,28.120051,0.408975,0.116569,0.357025,0.117431,0.000000,1203.000000,0.000099,0.071400,0.577100,0.00620,0.0,0.279300,0.063000,0.003000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6543.0,2022-07-24,0,114.303181,0.383183,0.143425,0.323636,0.149761,0.000180,2859.923089,0.000550,0.147657,0.345253,0.38769,0.0,0.011303,0.073058,0.035039
15697.0,2019-07-23,0,463.010280,0.434444,0.130950,0.328050,0.106556,0.091744,852.446647,0.000235,0.000400,0.571500,0.24750,0.0,0.000000,0.180600,0.000000
15680.0,2020-08-05,0,306.004903,0.470794,0.132175,0.277175,0.119856,0.002794,2197.943859,0.000522,0.114800,0.324200,0.53600,0.0,0.000000,0.024500,0.000500


In [23]:
temp_file

,2008010,2008011,2008012,2008013,2008014,2008015,2008016,2008017,2008018,2008019,...,20231221,20231222,20231223,20231224,20231225,20231226,20231227,20231228,20231229,20231230
id,,,,,,,,,,,,,,,,,,,,,
42,5.534027,1.250092,1.171051,7.654266,8.143341,6.858734,9.243744,8.367828,8.352570,9.129761,...,10.641083,10.621246,12.226654,12.117035,9.785583,10.418793,11.137787,10.728119,9.480316,9.080719
43,5.534027,1.250092,1.171051,7.654266,8.143341,6.858734,9.243744,8.367828,8.352570,9.129761,...,10.641083,10.621246,12.226654,12.117035,9.785583,10.418793,11.137787,10.728119,9.480316,9.080719
44,5.444183,1.140717,1.020660,7.615204,8.033966,6.786468,9.355072,8.219391,8.387726,9.014526,...,10.535614,10.705231,12.304779,12.191254,9.865662,10.330902,11.143646,10.843353,9.410004,9.033844
45,5.270355,1.029388,0.946442,7.425751,7.893341,6.735687,9.392181,8.113922,8.139679,8.815308,...,10.412567,10.609528,12.234467,12.015472,9.830505,10.151215,10.934662,10.595306,9.195160,8.891266
46,5.131683,0.972748,0.901520,7.251923,7.783966,6.749359,9.464447,8.043610,7.952179,8.659058,...,10.381317,10.603668,12.212982,11.904144,9.898865,10.063324,10.786224,10.454681,9.066254,8.807282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34121,3.789887,0.777435,0.967926,-6.824249,-4.227753,1.745453,3.208588,1.332672,0.858429,3.051636,...,8.705536,5.689606,6.283295,12.925629,13.910583,13.174652,9.969818,10.235931,6.704925,10.172516
34122,4.110199,1.623138,1.637848,-6.082062,-3.645721,1.991547,3.925384,2.449860,1.428742,3.172729,...,8.764130,5.857575,6.277435,12.499847,12.633240,12.694183,9.717865,9.771088,6.765472,9.432282
34123,4.110199,1.623138,1.637848,-6.082062,-3.645721,1.991547,3.925384,2.449860,1.428742,3.172729,...,8.764130,5.857575,6.277435,12.499847,12.633240,12.694183,9.717865,9.771088,6.765472,9.432282


In [24]:
temps = []
winds = []
rhums = []
prcps = []
temps_mean = []
prcps_sum = []
prcps_long_sum = []

for i, row in df_all.iterrows():
    
    year = str(row.date.year)
    month = str(row.date.month) 
    if len(month) < 2:
        month = str(0) + str(row.date.month)

    try:
        temps.append(temp_file.loc[[i], [year + month + str(int(row.date.day) -1) ]].values[0][0])
        temps_mean.append(temp_file_mean.loc[[i], [year + month + str(int(row.date.day) -1) ]].values[0][0])
        winds.append(wind_file.loc[[i], [year + month + str(int(row.date.day) -1) ]].values[0][0])
        rhums.append(rhum_file.loc[[i], [year + month + str(int(row.date.day) -1) ]].values[0][0])
        prcps.append(prcp_file.loc[[i], [year + month + str(int(row.date.day) -1) ]].values[0][0])
        prcps_sum.append(prcp_file_sum.loc[[i], [year + month + str(int(row.date.day) -1) ]].values[0][0])
        prcps_long_sum.append(prcp_file_long_sum.loc[[i], [year + month + str(int(row.date.day) -1) ]].values[0][0])
        # the days start from 0, hence the you need the -1 when you look up for the corresponding date
    except Exception as e:
        print(e)
        temps.append(np.nan)
        temps_mean.append(np.nan)
        winds.append(np.nan)
        rhums.append(np.nan)
        prcps.append(np.nan)
        prcps_sum.append(np.nan)
        prcps_long_sum.append(np.nan)

In [25]:

df_all['temp'] = temps
df_all['temp_mean_7day'] = temps_mean
df_all['wind'] = winds
df_all['rhum'] = rhums
df_all['prcp'] = prcps
df_all['prcp_sum_7day'] = prcps_sum
df_all['prcp_sum_28day'] = prcps_long_sum


In [26]:
# Check if there are any nan values
df_all[df_all.isna().any(axis=1)]

,date,ignit,mean_elev,north,east,south,west,slope30,road_den,pline_den,...,shrubland,grassland,other,temp,temp_mean_7day,wind,rhum,prcp,prcp_sum_7day,prcp_sum_28day
id,,,,,,,,,,,,,,,,,,,,,
14560.0,2020-07-24,0,4.728852,0.0,0.0,1.0,0.0,0.582889,0.0,0.0,...,NaN,NaN,NaN,24.292389,23.616887,6.889786,80.174866,0.001722,0.004453,0.057963


In [27]:
df_all = df_all.fillna(0)

In [28]:
df_all

,date,ignit,mean_elev,north,east,south,west,slope30,road_den,pline_den,...,shrubland,grassland,other,temp,temp_mean_7day,wind,rhum,prcp,prcp_sum_7day,prcp_sum_28day
id,,,,,,,,,,,,,,,,,,,,,
3198.0,2016-09-13,1,62.071857,0.405813,0.105250,0.347762,0.141175,0.000013,1926.000000,0.000321,...,0.290800,0.045400,0.028500,30.080353,25.287497,11.834545,46.936417,0.000002,0.000331,0.249933
3198.0,2018-09-17,1,62.071857,0.405813,0.105250,0.347762,0.141175,0.000013,1926.000000,0.000321,...,0.290800,0.045400,0.028500,23.805054,20.483425,8.637403,43.126170,0.000000,0.006399,0.460110
3365.0,2019-07-30,1,79.004539,0.381919,0.111675,0.370962,0.135444,0.000106,1807.000000,0.000431,...,0.217900,0.009100,0.013100,25.522370,28.786146,14.535752,46.038418,0.000001,0.030568,0.273808
3859.0,2022-08-31,1,31.601013,0.377481,0.127556,0.348756,0.146206,0.000000,1506.000000,0.000292,...,0.002600,0.065800,0.180500,21.569977,23.155191,16.873781,56.378426,0.000392,0.000924,0.240471
4014.0,2018-07-15,1,28.120051,0.408975,0.116569,0.357025,0.117431,0.000000,1203.000000,0.000099,...,0.279300,0.063000,0.003000,25.449127,22.238329,8.360366,36.902447,0.000002,0.002489,0.092618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6543.0,2022-07-24,0,114.303181,0.383183,0.143425,0.323636,0.149761,0.000180,2859.923089,0.000550,...,0.011303,0.073058,0.035039,29.175934,29.677713,3.427331,39.860860,0.000537,0.015666,0.344810
15697.0,2019-07-23,0,463.010280,0.434444,0.130950,0.328050,0.106556,0.091744,852.446647,0.000235,...,0.000000,0.180600,0.000000,28.402252,25.300201,5.958509,41.162640,0.006697,0.019545,0.941452
15680.0,2020-08-05,0,306.004903,0.470794,0.132175,0.277175,0.119856,0.002794,2197.943859,0.000522,...,0.000000,0.024500,0.000500,17.164703,23.009988,8.141712,77.010185,0.024511,0.073499,0.699566


In [29]:
df_all

,date,ignit,mean_elev,north,east,south,west,slope30,road_den,pline_den,...,shrubland,grassland,other,temp,temp_mean_7day,wind,rhum,prcp,prcp_sum_7day,prcp_sum_28day
id,,,,,,,,,,,,,,,,,,,,,
3198.0,2016-09-13,1,62.071857,0.405813,0.105250,0.347762,0.141175,0.000013,1926.000000,0.000321,...,0.290800,0.045400,0.028500,30.080353,25.287497,11.834545,46.936417,0.000002,0.000331,0.249933
3198.0,2018-09-17,1,62.071857,0.405813,0.105250,0.347762,0.141175,0.000013,1926.000000,0.000321,...,0.290800,0.045400,0.028500,23.805054,20.483425,8.637403,43.126170,0.000000,0.006399,0.460110
3365.0,2019-07-30,1,79.004539,0.381919,0.111675,0.370962,0.135444,0.000106,1807.000000,0.000431,...,0.217900,0.009100,0.013100,25.522370,28.786146,14.535752,46.038418,0.000001,0.030568,0.273808
3859.0,2022-08-31,1,31.601013,0.377481,0.127556,0.348756,0.146206,0.000000,1506.000000,0.000292,...,0.002600,0.065800,0.180500,21.569977,23.155191,16.873781,56.378426,0.000392,0.000924,0.240471
4014.0,2018-07-15,1,28.120051,0.408975,0.116569,0.357025,0.117431,0.000000,1203.000000,0.000099,...,0.279300,0.063000,0.003000,25.449127,22.238329,8.360366,36.902447,0.000002,0.002489,0.092618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6543.0,2022-07-24,0,114.303181,0.383183,0.143425,0.323636,0.149761,0.000180,2859.923089,0.000550,...,0.011303,0.073058,0.035039,29.175934,29.677713,3.427331,39.860860,0.000537,0.015666,0.344810
15697.0,2019-07-23,0,463.010280,0.434444,0.130950,0.328050,0.106556,0.091744,852.446647,0.000235,...,0.000000,0.180600,0.000000,28.402252,25.300201,5.958509,41.162640,0.006697,0.019545,0.941452
15680.0,2020-08-05,0,306.004903,0.470794,0.132175,0.277175,0.119856,0.002794,2197.943859,0.000522,...,0.000000,0.024500,0.000500,17.164703,23.009988,8.141712,77.010185,0.024511,0.073499,0.699566


In [30]:
df_all.to_csv(savepath + '/df_all_'+ season_name + '.csv')